In [1]:
import numpy as np
import scipy
import pylab as mplot
from scipy import interpolate as intrp
from scipy import integrate as intg

mplot.rc('text', usetex=True)
mplot.rc('font', family='serif')
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# add NFW with same mass
h0 = 0.7
Om0 = 0.3
z_max = 40./h0
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=100*h0, Om0=Om0)
from astropy import units as u

def mass_richness_sv(richness,z):
    #From SV paper
    M_0 = 2.35e14
    lambda_0 = 30.
    z_0 = 0.5
    F = 1.12
    G = 0.11
    #No factors of h
    M200m = M_0*((richness/lambda_0)**F)*((1.+z)/(1.+z_0))**G
    return M200m 

def mass_to_R200m(mass, z, cosmo):
    rho_m_z = cosmo.Om(z)*cosmo.critical_density(z)
    mass_units = mass*u.Msun
    R200m = ((mass_units/(200.*4.*np.pi*rho_m_z/3.))**(1./3.)).to('Mpc')
    return R200m.value

def m200_to_c200_duffy(mass_200, z, h, mean_or_crit_input):
    #Using full (not relaxed)                                                                                                                       
    M_pivot = (2.0e12)/h
    if (mean_or_crit_input == 'crit'):
        A = 5.71
        B = -0.084
        C = -0.47
    if (mean_or_crit_input == 'mean'):
        A = 10.14
        B = -0.081
        C = -1.01
    c_200 = A*((mass_200/M_pivot)**B)*(1.+z)**C
    return c_200

def rho_to_sigma_fast(rr, rho, R_array, h0):
    sigma = np.zeros(len(R_array))
    lnrho_func = intrp.interp1d(np.log(rr), np.log(rho))
    if (1): 
        minz_to_integrate = 0.0
        maxz_to_integrate = z_max
        num_z_tointegrate = 1500
        z_tointegrate = np.linspace(minz_to_integrate, maxz_to_integrate, num = num_z_tointegrate)
        for ri in range(0,len(R_array)):
            func_evals = np.exp(lnrho_func(np.log(np.sqrt(R_array[ri]**2. + z_tointegrate**2.))))
            #twice since integral is symmetric and want -infty to +infty
            sigma[ri] = 2.*intg.simps(func_evals, z_tointegrate)
    return sigma

def Sigma_to_DeltaSigma_fast(R_output, R_Sigma, Sigma, params, minR):

    temp = np.copy(R_Sigma)
    temp[0] *= 0.999999
    temp[-1] *= 1.000001
    sigma_interp_func = intrp.interp1d(np.log(temp), Sigma)
    to_integrate_func = intrp.interp1d(temp, Sigma*R_Sigma)

    dsig = np.zeros(len(R_output))
    for i in range(len(R_output)):
        if (minR < np.min(temp) or np.min(R_output) < np.min(temp)): 
            pdb.set_trace()
        integral_result = intg.quad(to_integrate_func, minR, R_output[i], epsabs=1.0e-03, epsrel=1.0e-03)
        dsig[i] = 2.*(integral_result[0] + params['integral_R_min'])/(R_output[i]**2.) - sigma_interp_func(np.log(R_output[i]))

    return dsig

def NFW(rho_0, r_s, rr):
    rho = (rho_0)/((rr/r_s)*(1.+rr/r_s)**2.)
    return rho

def gNFW(rho_0, r_s, alpha, beta, gamma, rr):
    rho = rho_0 * (rr/r_s)**(-gamma)*(1.+(rr/r_s)**alpha)**((gamma-beta)/alpha)
    return rho

def gNFW_delta(rho_0, r_s, alpha, beta, gamma, r_sh, delta, rr):
    rho = rho_0 * (rr/r_s)**(-gamma)*(1.+(rr/r_s)**alpha)**((gamma-beta)/alpha)
    rho[rr>=r_sh] *= (rr[rr>=r_sh]/r_sh)**(-delta)     
    return rho

def gNFW_sh(rho_0, r_s, alpha, beta, gamma, r_sh, delta, Q_sh, rr):
    rho = rho_0 * (rr/r_s)**(-gamma)*(1.+(rr/r_s)**alpha)**((gamma-beta)/alpha)
    rho[rr>=r_sh] *= (rr[rr>=r_sh]/r_sh)**(-delta)*Q_sh        
    return rho

In [5]:
rr = np.exp(np.linspace(np.log(0.0001), np.log(100.0), num = 10000)) # Comoving Mpc
RR = np.exp(np.linspace(np.log(0.01), np.log(50.0), num = 100)) # Comoving Mpc

def sigma_gNFW_sh(lam, z, alpha, beta, gamma, r_sh, delta, Q_sh, rr, RR, h0):
    M_200 = mass_richness_sv(lam, z)

    #Physical
    R_200m = mass_to_R200m(M_200, z, cosmo)
    c_200m = m200_to_c200_duffy(M_200, z, h0, 'mean')
    c_factor = np.log(1.+c_200m) - c_200m/(1.+c_200m)

    #Comoving
    r_s = (1.+z)*R_200m/c_200m 
    rho_0 = M_200/(4.*np.pi*c_factor*r_s**3.) # Msun/Mpc^3 comoving

    rho_gNFW_sh = gNFW_sh(rho_0, r_s, alpha, beta, gamma, r_sh, delta, Q_sh, rr)
    sigma_gNFW_sh = rho_to_sigma_fast(rr, rho_gNFW_sh, RR, h0)

In [ ]:
def priors(params, h0, splash):

    ln_alpha, ln_beta, ln_gamma, ln_r_s, ln_r_t, ln_rho_O, ln_rho_s, se = params
    beta = 10**ln_beta
    gamma = 10**ln_gamma
    r_t = 10**ln_r_t

    alpha = 10**ln_alpha
    r_s = 10**ln_r_s
    rho_O = 10**ln_rho_O
    rho_s = 10**ln_rho_s  

    min_logrs = np.log10(0.1/h0)
    max_logrs = np.log10(5.0/h0)
    min_logrt = np.log10(0.1/h0)
    max_logrt = np.log10(5.0/h0)
    min_se = -10.0
    max_se = 10.0
    
    if (ln_r_s > max_logrs) or (ln_r_s < min_logrs) or (ln_r_t > max_logrt) or (ln_r_t < min_logrt)  or (se < min_se) or (se > max_se):
        lnprior = -1.0e10

    else:
        mean_logalpha = np.log10(0.2)
        sigma_logalpha = 0.6
        mean_logbeta = np.log10(4.)
        sigma_logbeta = 0.2
        mean_loggamma = np.log10(6.0)
        sigma_loggamma = 0.2

        lnprior_alpha = -0.5*np.log(2.*np.pi*sigma_logalpha**2.)-0.5*((ln_alpha - mean_logalpha)**2.)/sigma_logalpha**2.
        lnprior_beta =  -0.5*np.log(2.*np.pi*sigma_logbeta**2.)-0.5*((ln_beta - mean_logbeta)**2.)/sigma_logbeta**2.
        lnprior_gamma =  -0.5*np.log(2.*np.pi*sigma_loggamma**2.)-0.5*((ln_gamma- mean_loggamma)**2.)/sigma_loggamma**2.
        lnprior = lnprior_alpha + lnprior_beta + lnprior_gamma

    return lnprior

In [ ]:

def lnlikelihood(params, R, z, data_vec, invcov, h0, splash):

    lnlike_priors = priors(params, h0, splash)
    lnlike_data = 0.0
    
    if (lnlike_priors > -1.0e5):
        
        model = sigma_gNFW_sh(lam, z, alpha, beta, gamma, r_sh, delta, Q_sh, rr, RR, h0)
        diff = data_vec - model
        detinvcov = np.linalg.det(invcov)
        detcov = 1./detinvcov
        lnlike_data = -0.5*(len(data_vec)*np.log(2.*np.pi) + np.log(detcov)) -0.5*np.dot(diff, np.dot(invcov, diff))
    
    lnlike = lnlike_data + lnlike_priors
       
    return lnlike


In [ ]:
params = []